In [1]:
# Importing necessary libraries for data manipulation and visualization
import sys
import os

sys.path.append(os.path.abspath(".."))
from data_handlers.main import DataHandler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium

In [2]:
data_csv = pd.read_csv("../data/flight_states.csv")
f_sts_df = pd.DataFrame(data_csv)

In [3]:
f_sts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46857 entries, 0 to 46856
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   icao24            46857 non-null  object 
 1   callsign          46100 non-null  object 
 2   origin_country    46857 non-null  object 
 3   time_position     46471 non-null  float64
 4   last_contact      46857 non-null  int64  
 5   last_contact_utc  46857 non-null  object 
 6   longitude         46471 non-null  float64
 7   latitude          46471 non-null  float64
 8   geo_altitude      42613 non-null  float64
 9   on_ground         46857 non-null  bool   
 10  velocity          46842 non-null  float64
 11  true_track        46857 non-null  float64
 12  vertical_rate     43050 non-null  float64
 13  sensors           0 non-null      float64
 14  baro_altitude     42857 non-null  float64
 15  squawk            25083 non-null  float64
 16  spi               46857 non-null  bool  

In [4]:
f_sts_df.isna().sum()

icao24                  0
callsign              757
origin_country          0
time_position         386
last_contact            0
last_contact_utc        0
longitude             386
latitude              386
geo_altitude         4244
on_ground               0
velocity               15
true_track              0
vertical_rate        3807
sensors             46857
baro_altitude        4000
squawk              21774
spi                     0
position_source         0
category                0
dtype: int64

In [5]:
if (
    "sensors" in f_sts_df.columns
    or "position_source" in f_sts_df.columns
    or "spi" in f_sts_df.columns
):
    f_sts_df = f_sts_df.drop("sensors", axis=1)
    f_sts_df = f_sts_df.drop("position_source", axis=1)
    f_sts_df = f_sts_df.drop("spi", axis=1)
else:
    pass

In [6]:
f_sts_df = f_sts_df[f_sts_df.category > 0]

In [7]:
f_sts_df.origin_country.value_counts()

origin_country
United States                 1750
Australia                      158
Germany                         97
Spain                           91
Austria                         89
United Kingdom                  88
France                          82
China                           74
Kingdom of the Netherlands      67
Ireland                         66
Belgium                         64
Canada                          60
Switzerland                     52
Turkey                          45
Republic of Korea               34
United Arab Emirates            29
Luxembourg                      27
Taiwan                          25
Qatar                           20
Denmark                         15
Estonia                         14
Portugal                        10
Israel                           9
Saudi Arabia                     9
Mexico                           9
Morocco                          9
Poland                           8
Egypt                            7
Chile

In [8]:
f_sts_df.callsign.value_counts()

callsign
            15
HBAL712      5
PAT239       4
AAL1632      4
JBU377       4
            ..
LXJ413       1
JIA5189      1
N6026Z       1
GFD5         1
SWR282       1
Name: count, Length: 2465, dtype: int64

In [9]:
f_sts_df.icao24.value_counts()

icao24
adc9b6    5
a01fee    5
45d064    5
ab2b23    5
adcd6d    5
         ..
a6c5dc    1
adfd6f    1
a05467    1
ac31a1    1
abc0c1    1
Name: count, Length: 1688, dtype: int64

In [10]:
f_sts_df.sample(n=5)

,icao24,callsign,origin_country,time_position,last_contact,last_contact_utc,longitude,latitude,geo_altitude,on_ground,velocity,true_track,vertical_rate,baro_altitude,squawk,category
37833,a3db5e,DAL2242,United States,1.728617e+09,1728616903,2024-10-11 03:21:43+00:00,-78.9392,36.0215,1866.90,False,148.68,246.98,8.13,1836.42,NaN,4
13861,3986e8,AFR16ZE,France,1.728597e+09,1728596912,2024-10-10 21:48:32+00:00,2.6382,48.9183,3009.90,False,162.07,89.09,-8.78,3002.28,1000.0,1
29650,a3a9b6,EDV5296,United States,1.728612e+09,1728611973,2024-10-11 01:59:33+00:00,-78.3527,42.9158,3276.60,False,154.52,99.78,9.10,3238.50,7476.0,4
5268,ae13ae,DRAGN41,United States,1.728597e+09,1728596908,2024-10-10 21:48:28+00:00,-98.2672,36.7133,2567.94,False,65.63,326.19,76.40,2392.68,NaN,13
40720,846333,NCA167,Japan,1.728617e+09,1728616904,2024-10-11 03:21:44+00:00,140.3988,35.7766,160.02,False,78.17,329.55,-4.55,106.68,3604.0,6


In [11]:
US_flights = pd.DataFrame(f_sts_df[f_sts_df.origin_country == "United States"])
US_flights.sample(n=5)
US_flights = US_flights.dropna(how="any")
US_flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 805 entries, 92 to 46816
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   icao24            805 non-null    object 
 1   callsign          805 non-null    object 
 2   origin_country    805 non-null    object 
 3   time_position     805 non-null    float64
 4   last_contact      805 non-null    int64  
 5   last_contact_utc  805 non-null    object 
 6   longitude         805 non-null    float64
 7   latitude          805 non-null    float64
 8   geo_altitude      805 non-null    float64
 9   on_ground         805 non-null    bool   
 10  velocity          805 non-null    float64
 11  true_track        805 non-null    float64
 12  vertical_rate     805 non-null    float64
 13  baro_altitude     805 non-null    float64
 14  squawk            805 non-null    float64
 15  category          805 non-null    int64  
dtypes: bool(1), float64(9), int64(2), object(4)
me

In [12]:
craft_categories = {
    0: {"No information at all": "gray"},
    1: {"No ADS-B Emitter Category Information": "black"},
    2: {"Light (< 15500 lbs)": "lightblue"},
    3: {"Small (15500 to 75000 lbs)": "green"},
    4: {"Large (75000 to 300000 lbs)": "orange"},
    5: {"High Vortex Large (aircraft such as B-757)": "red"},
    6: {"Heavy (> 300000 lbs)": "darkred"},
    7: {"High Performance (> 5g acceleration and 400 kts)": "purple"},
    8: {"Rotorcraft": "lightgreen"},
    9: {"Glider / sailplane": "darkgreen"},
    10: {"Lighter-than-air": "cadetblue"},
    11: {"Parachutist / Skydiver": "pink"},
    12: {"Ultralight / hang-glider / paraglider": "lightyellow"},
    13: {"Reserved": "gray"},
    14: {"Unmanned Aerial Vehicle": "lightgray"},
    15: {"Space / Trans-atmospheric vehicle": "beige"},
    16: {"Surface Vehicle – Emergency Vehicle": "red"},
    17: {"Surface Vehicle – Service Vehicle": "darkblue"},
    18: {"Point Obstacle (includes tethered balloons)": "blue"},
    19: {"Cluster Obstacle": "darkpurple"},
    20: {"Line Obstacle": "darkgray"},
}

In [19]:
def add_to_map(df):
    # location=[39.8283, -98.5795] >> replace this with the lon lat of the origin country
    map = folium.Map(
        location=(0, 0),
        zoom_control=True,
        zoom_start=2,
        tiles=folium.TileLayer(no_wrap=True),
    )
    feature_groups = {}

    for index, row in df.iterrows():
        if pd.notna(row["longitude"]) and pd.notna(row["latitude"]):

            category = row["category"]

            if category not in craft_categories:
                description = "Unknown"
                color = "grey"
            else:
                description, color = list(craft_categories[category].items())[0]

            if description not in feature_groups:
                feature_groups[description] = folium.FeatureGroup(
                    name=description, show=True
                )

            folium.Marker(
                location=[row["latitude"], row["longitude"]],
                tooltip="📝",
                popup=description,
                icon=folium.Icon(icon="plane", color=color),
            ).add_to(feature_groups[description])

    for feature in feature_groups.values():
        feature.add_to(map)

    folium.LayerControl().add_to(map)
    map.save("../maps_html/all_crafts.html")


add_to_map(US_flights)

In [20]:
# maximum_flyer = str(f_sts_df.icao24.value_counts().head(1))
# print(maximum_flyer)
icao24_craft = pd.DataFrame(f_sts_df[f_sts_df.icao24 == "adc9b6"])
icao24_craft

,icao24,callsign,origin_country,time_position,last_contact,last_contact_utc,longitude,latitude,geo_altitude,on_ground,velocity,true_track,vertical_rate,baro_altitude,squawk,category
1183,adc9b6,JBU700,United States,1.728542e+09,1728541784,2024-10-10 06:29:44+00:00,-118.4048,33.9405,NaN,True,0.00,81.56,NaN,NaN,NaN,4
7541,adc9b6,JBU377,United States,1.728597e+09,1728596911,2024-10-10 21:48:31+00:00,-80.9735,26.5662,5989.32,False,192.62,310.67,7.15,5715.00,7442.0,4
17987,adc9b6,JBU377,United States,1.728602e+09,1728601911,2024-10-10 23:11:51+00:00,-91.9876,29.9511,11612.88,False,205.56,294.71,0.00,10972.80,7442.0,4
28615,adc9b6,JBU377,United States,1.728612e+09,1728611975,2024-10-11 01:59:35+00:00,-108.0026,35.2937,11498.58,False,237.66,279.22,0.00,10980.42,2461.0,4
39118,adc9b6,JBU377,United States,1.728617e+09,1728616903,2024-10-11 03:21:43+00:00,-119.0358,37.5327,12108.18,False,204.25,279.13,0.33,11582.40,2461.0,4


In [22]:
def track_flight_icao24(craft_df):
    poly_colors = [
        "green",
        "blue",
        "purple",
        "orange",
        "red",
        "darkgreen",
        "darkred",
        "lightblue",
        "lightgreen",
        "pink",
        "beige",
        "lightred",
        "gray",
        "black",
        "cadetblue",
        "lightgray",
        "darkblue",
        "darkpurple",
        "white",
    ]

    origin_lat = craft_df.iloc[0].latitude
    origin_lon = craft_df.iloc[0].longitude
    way_points = []

    map = folium.Map(
        location=[origin_lat, origin_lon],
        zoom_start=4,
        tiles=folium.TileLayer(no_wrap=True),
    )
    for index, row in craft_df.iterrows():

        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            tooltip="📝",
            popup="Flight Path",
            icon=folium.Icon(icon="plane", color=poly_colors[index % len(poly_colors)]),
        ).add_to(map)

        way_points.append([row["latitude"], row["longitude"]])

    for point in range(len(way_points) - 1):
        folium.PolyLine(
            (way_points[point], way_points[point + 1]),
            color=poly_colors[point % len(poly_colors)],
            weight=2.5,
            opacity=1,
        ).add_to(map)

    map.save("../maps_html/icao24_flight_path.html")


track_flight_icao24(icao24_craft)